In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
!pip install pydicom

In [4]:
import copy
from datetime import timedelta, datetime
import imageio
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing
import numpy as np
import os
from pathlib import Path
import pydicom
import pytest
import scipy.ndimage as ndimage
from scipy.ndimage.interpolation import zoom
from skimage import measure, morphology, segmentation
from time import time, sleep
from tqdm import trange
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DistributedSampler, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import warnings
import glob

<ipython-input-4-9cc11f3a9ccf>:13: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import zoom


In [5]:
class CTScansDataset(Dataset):
    def __init__(self, root_dir, transform=None,transform2=None):
        self.root_dir = Path(root_dir)
        self.patients = [p for p in glob.glob(root_dir+'*')]
        self.transform = transform
        self.transform2 = transform2

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image, metadata = self.load_scan(self.patients[idx])
        sample = {'image': image, 'metadata': metadata}
        return sample

    def get_patient(self, patient_id):
        patient_ids = [str(p.stem) for p in self.patients]
        return self.__getitem__(patient_ids.index(patient_id))

    @staticmethod
    def load_scan(path):
        T = [row for row in os.listdir(path)]
        slices = [pydicom.dcmread(path + "/" + file) for file in T]
        try:
            slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        except:
            pass
        images = np.stack([s.pixel_array.astype(float) for s in slices])
        images = images.astype(np.int16)
        for n in range(len(slices)):
            intercept = slices[n].RescaleIntercept
            slope = slices[n].RescaleSlope
            if slope != 1:
                images[n] = slope * images[n].astype(np.float64)
                images[n] = images[n].astype(np.int16)
            images[n] += np.int16(intercept)
        image = images
        if image.shape[1]!=512 or image.shape[2]!=512:
            mask = image>image[0,0,0]
            z,y,x = np.where(mask)
            z_min,z_max,y_min,y_max,x_min,x_max = z.min(),z.max(),y.min(),y.max(),x.min(),x.max()
            image = image[:,y_min:y_max,x_min:x_max]
        return image, slices[0]

In [6]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_train_data/'

In [7]:
from math import sqrt
from joblib import Parallel, delayed
train = CTScansDataset(root_dir=train_dir)

In [8]:
# for i in range(len(train)):
#   try:
#     print(train[i]["metadata"])
#   except Exception:
#     continue

In [9]:
!cp "/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/notebooks/segmentation/Resnet34Unet.py" ./

In [10]:
import importlib
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import numpy as np
import torch
import os
import pandas as pd
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import cv2
from PIL import Image
import glob
import os
from Resnet34Unet import ResnetSuperVision

In [11]:
import os
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader,Dataset
import albumentations as albu
#from albumentations import torch as AT
from skimage.color import gray2rgb




def get_img(path,x, folder: str='train_images'):
    """
    Return image based on image name and folder.
    """
    data_folder = os.path.join(path,folder)
    image_path = os.path.join(data_folder, x)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def to_tensor(x, **kwargs):
    """
    Convert image or mask.
    """
    return x.transpose(2, 0, 1).astype('float32')


sigmoid = lambda x: 1 / (1 + np.exp(-x))


def post_process(probability, threshold, min_size):
    """
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num


def get_training_augmentation(y=704,x=1024):
    train_transform = [albu.RandomBrightnessContrast(p=0.3),
#                            albu.RandomGamma(p=0.3),
                           albu.VerticalFlip(p=0.5),
                           albu.HorizontalFlip(p=0.5),
#                            albu.ShiftScaleRotate(scale_limit=0, rotate_limit=10, shift_limit=0.0625, p=0.5, border_mode=0),
                           albu.Downscale(p=1.0,scale_min=0.35,scale_max=0.75,),
                           albu.Resize(y, x),
                           albu.RandomCrop(height=400, width=400,p=0.5),
#                            albu.GridDistortion(p=0.5),
#                            albu.OpticalDistortion(p=0.5, distort_limit=2, shift_limit=0.5),
                           albu.Resize(y, x)]
    return albu.Compose(train_transform)


def get_validation_augmentation(y=704,x=1024):
    """Add paddings to make image shape divisible by 32"""
    test_transform = [albu.Resize(y, x)]
    return albu.Compose(test_transform)


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)


def set_lungwin(img, hu=[-1200., 600.]):
    lungwin = np.array(hu)
    newimg = (img-lungwin[0]) / (lungwin[1]-lungwin[0])
    newimg[newimg < 0] = 0
    newimg[newimg > 1] = 1
    return newimg

class CTDataset2D(Dataset):
    def __init__(self, x_train,y_train,y_train2,transforms = albu.Compose([albu.HorizontalFlip()]),preprocessing=None,size=512):
        self.x_train = x_train
        self.y_train = y_train
        self.y_train2 = y_train2
        self.transforms = transforms
        self.preprocessing = preprocessing
        self.size=size
        self.hu =[[-1200., 600.],[-1800., 1000.],[-1500., 800.]]

    def __getitem__(self, idx):
        img = self.x_train[idx]
        mask = self.y_train[idx]
        mask2 = self.y_train2[idx]
        mask = np.expand_dims(mask,axis=-1)
        mask2 = np.expand_dims(mask2,axis=-1)
        mask = np.concatenate([mask,mask2],axis=-1)
        img = set_lungwin(img,self.hu[np.random.randint(3)])
        img = img*255
        img = img.astype(np.uint8)
        img = gray2rgb(img)

        augmented = self.transforms(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        if self.preprocessing:
            preprocessed = self.preprocessing(image=img, mask=mask)
            img = preprocessed['image']
            mask = preprocessed['mask']
        if self.y_train2[idx].sum()==0:
            label = np.array(0.0)
        else:
            label = np.array(self.y_train[idx].sum()/self.y_train2[idx].sum())
        return img, mask,torch.from_numpy(label.reshape(1,1))

    def __len__(self):
        return len(self.x_train)



def norm(img):
    img-=img.min()
    return img/img.max()


In [12]:
import numpy as np
import functools
def preprocess_input(
    x, mean=None, std=None, input_space="RGB", input_range=None, **kwargs
):

    if input_space == "BGR":
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.0

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x

In [13]:
formatted_settings = {
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],}

In [14]:
class config:
    model_name="resnet34"
    batch_size = 1
    WORKERS = 1
    DataLoder = {}
    DataLoder['PY'] = ""
    DataLoder["CLASS"] = "CTDataset2D"
    DataLoder["x_size"] = 512
    DataLoder["y_size"] = 512
    DataLoder["margin"] = 512
    preprocessing_fn = functools.partial(preprocess_input, **formatted_settings)
    seg_classes =2
    resume = True
    MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/notebooks/segmentation/saved_model_weights'
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)

In [15]:
def get_train_loder(x,y,y2):
    train_dataset = CTDataset2D(x, y,y2,
                                  transforms=get_training_augmentation(x=config.DataLoder["x_size"], y=config.DataLoder["y_size"]),
                                  preprocessing=get_preprocessing(config.preprocessing_fn))
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=4, pin_memory=True)
    return train_loader,train_dataset
def get_val_loder(x,y,y2,b=18):
    val_dataset = CTDataset2D(x, y,y2,
                                  transforms=get_validation_augmentation(x=config.DataLoder["x_size"], y=config.DataLoder["y_size"]),
                                  preprocessing=get_preprocessing(config.preprocessing_fn))
    valid_loader = DataLoader(val_dataset, batch_size=b, shuffle=False, num_workers=config.WORKERS, pin_memory=True)
    return valid_loader,val_dataset

In [16]:
model = ResnetSuperVision(config.seg_classes, backbone_arch='resnet34').cuda()

In [17]:
# model.load_state_dict(torch.load("../input/osic-model/resnet34_fib_best.pth"))

In [18]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
class trainer:
    def __init__(self,model):
        self.model = model
        if config.resume==True:
            self.load_best_model()

    def batch_valid(self, batch_imgs,get_fet):
        self.model.train()
        batch_imgs = batch_imgs.cuda()
        with torch.no_grad():
            predicted = self.model(batch_imgs,get_fet)
        predicted2 = torch.sigmoid(predicted[0])
        if not get_fet:
            return predicted2.cpu().numpy().astype(np.float32),predicted[1].cpu().numpy().astype(np.float32)
        else:
            return predicted2.cpu().numpy().astype(np.float32),predicted[1].cpu().numpy().astype(np.float32),predicted[2].cpu().numpy().astype(np.float32)

    def load_best_model(self):
        if os.path.exists(config.MODEL_PATH+"/{}_fib_best.pth".format(config.model_name)):
            self.model.load_state_dict(torch.load(config.MODEL_PATH+"/{}_fib_best.pth".format(config.model_name)))

    def predict(self,imgs_tensor,get_fet = False):
        self.model.train()
        with torch.no_grad():
            return self.batch_valid(imgs_tensor,get_fet=get_fet)


In [19]:
Trainer = trainer(model)

In [20]:
Trainer.load_best_model()

In [21]:
# 'ID00009637202177434476278', 'ID00010637202177584971671', 'ID00015637202177877247924', 'ID00317637202283194142136', 'ID00358637202295388077032'

## Test

In [22]:
imgs_path = '/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_train_data/'
targets_path = "/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_png_train_mask_data/"

In [ ]:
for patient_id in os.listdir(imgs_path):
  print(patient_id)
  # if not os.path.exists(targets_path + patient_id):
  #   os.makedirs(targets_path + patient_id)
  if patient_id == "ID00108637202209619669361":
    test_dir = imgs_path + patient_id
    test_bad = CTScansDataset(root_dir=test_dir)
    shape = len(os.listdir(test_dir))
    for ids in tqdm(range(len(test_bad))):
        scan = test_bad[ids]['image']
        # name = train[ids]['metadata']
        center = len(scan)//2
        test = scan
        val_loder2,_ = get_val_loder(np.array(test),np.array(test),np.array(test),b=1)
        for i,(imgs,masks,label) in enumerate(val_loder2):
            if True:
                res = Trainer.predict(imgs.cuda(),get_fet=True)
                res1 = Trainer.predict(torch.flip(imgs.cuda(),[-1]),get_fet=True)
                res2 = Trainer.predict(torch.flip(imgs.cuda(),[-2]),get_fet=True)
                res = list(res)
                res[0] = np.flip(res1[0],-1)+ res[0] +np.flip(res2[0],-2)
                res[1] = res1[1]+ res[1] +res2[1]
                res[0] = res[0] /3
                res[1] = res[1] /3
                res[2] = res1[2]+ res[2] +res2[2]
                res[2] = res[2] /3
                lung_mask = res[0][0,1]>0.9
                lung_mask = cv2.resize(lung_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
                # fibrosis_mask = res[0][0,0]>0.9
                # fibrosis_mask = cv2.resize(fibrosis_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))

                im = norm(imgs.cpu().numpy()[0].transpose(1,2,0))
                im = im.astype(float)
                rescaled_image = (np.maximum(im,0)/im.max())*255 # float pixels
                final_image = np.uint8(rescaled_image) # integers pixels
                final_image = Image.fromarray(final_image)

                mask = lung_mask.astype(float)
                rescaled_mask = (np.maximum(mask,0)/mask.max())*255 # float pixels
                final_mask = np.uint8(rescaled_mask) # integers pixels
                final_mask = Image.fromarray(final_mask)
                final_mask.save(f"{targets_path}{patient_id}/{shape - i}.png")

ID00392637202302319160044
ID00398637202303897337979
ID00393637202302431697467
ID00400637202305055099402
ID00401637202305320178010
ID00407637202308788732304
ID00405637202308359492977
ID00408637202308839708961
ID00417637202310901214011
ID00072637202198161894406
ID00076637202199015035026
ID00090637202204766623410
ID00093637202205278167493
ID00360637202295712204040
ID00355637202295106567614
ID00371637202296828615743
ID00370637202296737666151
ID00378637202298597306391
ID00364637202296074419422
ID00368637202296470751086
ID00376637202297677828573
ID00365637202296085035729
ID00367637202296290303449
ID00381637202299644114027
ID00383637202300493233675
ID00388637202301028491611
ID00422637202311677017371
ID00421637202311550012437
ID00423637202312137826377
ID00414637202310318891556
ID00426637202313170790466
ID00419637202311204720264
ID00411637202309374271828
ID00007637202177411956430
ID00011637202177653955184
ID00014637202177757139317
ID00012637202177665765362
ID00027637202179689871102
ID0002063720

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
# ID00052637202186188008618 --> 311, ID00078637202199415319443 --> 1k ,ID00108637202209619669361 --> 512 , ID00173637202238329754031 --> 600,ID00291637202279398396106 --> 484, ID00309637202282195513787 --> 469

## Test_

In [ ]:
test_dir_bad = '/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_train_data/ID00009637202177434476278'
test_bad = CTScansDataset(root_dir=test_dir_bad)

In [ ]:
for ids in tqdm(range(len(test_bad))):
    scan = test_bad[ids]['image']
    # name = train[ids]['metadata']

    center = len(scan)//2
    test = scan
    val_loder2,_ = get_val_loder(np.array(test),np.array(test),np.array(test),b=1)


    for i,(imgs,masks,label) in enumerate(val_loder2):
        if True:
            res = Trainer.predict(imgs.cuda(),get_fet=True)
            res1 = Trainer.predict(torch.flip(imgs.cuda(),[-1]),get_fet=True)
            res2 = Trainer.predict(torch.flip(imgs.cuda(),[-2]),get_fet=True)
            res = list(res)
            res[0] = np.flip(res1[0],-1)+ res[0] +np.flip(res2[0],-2)
            res[1] = res1[1]+ res[1] +res2[1]
            res[0] = res[0] /3
            res[1] = res[1] /3
            res[2] = res1[2]+ res[2] +res2[2]
            res[2] = res[2] /3
            lung_mask = res[0][0,1]>0.9
            lung_mask = cv2.resize(lung_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
            # fibrosis_mask = res[0][0,0]>0.9
            # fibrosis_mask = cv2.resize(fibrosis_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))

            im = norm(imgs.cpu().numpy()[0].transpose(1,2,0))
            im = im.astype(float)
            rescaled_image = (np.maximum(im,0)/im.max())*255 # float pixels
            final_image = np.uint8(rescaled_image) # integers pixels
            final_image = Image.fromarray(final_image)

            mask = lung_mask.astype(float)
            rescaled_mask = (np.maximum(mask,0)/mask.max())*255 # float pixels
            final_mask = np.uint8(rescaled_mask) # integers pixels
            final_mask = Image.fromarray(final_mask)

            final_mask.save(f"/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_png_train_mask_data/ID00009637202177434476278/{shape - i}.png")

            # final_mask.save(f"/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_png_train_mask_data/ID00009637202177434476278/{shape - i}.png")
            # final_image.save(f"/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_png_train_mask_data/ID00358637202295388077032/{shape - i}_img.png")
            # plt.figure(figsize=[20,10])
            # plt.subplot(131)
            # plt.title("original slide")
            # plt.imshow(im,cmap='gray')
            # plt.subplot(132)
            # plt.imshow(im,cmap='gray')
            # plt.imshow(res[0][0,1]>0.9,cmap='gray',alpha=.5)
            # plt.title("lung segmentation")
            # plt.subplot(133)
            # # plt.imshow(im)
            # plt.imshow(lung_mask,cmap='gray')
            # plt.title("lung mask")
            # plt.show()


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-61-1551fe7fcef8>:34: RuntimeWarning: invalid value encountered in divide
  rescaled_mask = (np.maximum(mask,0)/mask.max())*255 # float pixels


In [ ]:
test_dir_good = '/content/drive/MyDrive/Colab Notebooks/Pulmonary Fibrosis/data/sample_train_data/ID00009637202177434476278'
test_good = CTScansDataset(root_dir=test_dir_good)

In [ ]:
for ids in tqdm(range(len(test_good))):
    scan = train[ids]['image']
    center = len(scan)//2
    test = scan[::5]
    val_loder2,_ = get_val_loder(np.array(test),np.array(test),np.array(test),b=1)
    for i,(imgs,masks,label) in enumerate(val_loder2):
        if True:
            res = Trainer.predict(imgs.cuda(),get_fet=True)
            res1 = Trainer.predict(torch.flip(imgs.cuda(),[-1]),get_fet=True)
            res2 = Trainer.predict(torch.flip(imgs.cuda(),[-2]),get_fet=True)
            res = list(res)
            res[0] = np.flip(res1[0],-1)+ res[0] +np.flip(res2[0],-2)
            res[1] = res1[1]+ res[1] +res2[1]
            res[0] = res[0] /3
            res[1] = res[1] /3
            res[2] = res1[2]+ res[2] +res2[2]
            res[2] = res[2] /3
            lung_mask = res[0][0,1]>0.9
            lung_mask = cv2.resize(lung_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
            # fibrosis_mask = res[0][0,0]>0.9
            # fibrosis_mask = cv2.resize(fibrosis_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
            im = norm(imgs.cpu().numpy()[0].transpose(1,2,0))
            plt.figure(figsize=[20,10])
            plt.subplot(131)
            plt.title("original slide")
            plt.imshow(im,cmap='gray')
            plt.subplot(132)
            plt.imshow(im,cmap='gray')
            plt.imshow(res[0][0,1]>0.9,cmap='gray',alpha=.5)
            plt.title("lung segmentation")
            plt.subplot(133)
            # plt.imshow(im)
            plt.imshow(lung_mask,cmap='gray')
            plt.title("lung mask")
            plt.show()

In [ ]:
for ids in tqdm(range(len(train))):
    scan = train[ids]['image']
    center = len(scan)//2
    test = scan[center-3:center+3]
    val_loder2,_ = get_val_loder(np.array(test),np.array(test),np.array(test),b=1)
    for i,(imgs,masks,label) in enumerate(val_loder2):
        if True:
            res = Trainer.predict(imgs.cuda(),get_fet=True)
            res1 = Trainer.predict(torch.flip(imgs.cuda(),[-1]),get_fet=True)
            res2 = Trainer.predict(torch.flip(imgs.cuda(),[-2]),get_fet=True)
            res = list(res)
            res[0] = np.flip(res1[0],-1)+ res[0] +np.flip(res2[0],-2)
            res[1] = res1[1]+ res[1] +res2[1]
            res[0] = res[0] /3
            res[1] = res[1] /3
            res[2] = res1[2]+ res[2] +res2[2]
            res[2] = res[2] /3
            lung_mask = res[0][0,1]>0.9
            lung_mask = cv2.resize(lung_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
            # fibrosis_mask = res[0][0,0]>0.9
            # fibrosis_mask = cv2.resize(fibrosis_mask.astype(np.uint8),(test[i].shape[1],test[i].shape[0]))
            im = norm(imgs.cpu().numpy()[0].transpose(1,2,0))
            plt.figure(figsize=[20,10])
            plt.subplot(131)
            plt.title("original slide")
            plt.imshow(im,cmap='gray')
            plt.subplot(132)
            plt.imshow(im,cmap='gray')
            plt.imshow(res[0][0,1]>0.9,cmap='gray',alpha=.5)
            plt.title("lung segmentation")
            plt.subplot(133)
            # plt.imshow(im)
            plt.imshow(lung_mask,cmap='gray')
            plt.title("lung mask")
            plt.show()

Output hidden; open in https://colab.research.google.com to view.